In [1]:
# !pip install scikit-surprise
# !pip install statsmodels


In [2]:
# Import the libraries

import pandas as pd
import joblib
import numpy as np
# import matplotlib.pyplot as plt
# import sklearn

# from surprise import SVD, Dataset, Reader
# from surprise.model_selection import train_test_split
# from sklearn.utils.fixes import signature

import recmetrics

In [3]:
#Input of rating and workflow. Merge

ratings = pd.read_csv('new rating.csv')
workflows = pd.read_csv('new workflow.csv')
ratings = pd.merge(workflows,ratings).drop(['userActivity','timestamp'],axis=1)
ratings

,workflow,workflowTitle,userId,rating
0,1,Web Browsing 1,1,4.0
1,1,Web Browsing 1,5,4.0
2,1,Web Browsing 1,7,4.5
3,1,Web Browsing 1,15,2.5
4,1,Web Browsing 1,17,4.5
...,...,...,...,...
100831,193581,Communication 1744,184,4.0
100832,193583,Communication 1745,184,3.5
100833,193585,Communication 1746,184,3.5
100834,193587,Communication 1747,184,3.5


In [4]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['workflowTitle'],values='rating')
user_ratings

workflowTitle,Communication 1,Communication 10,Communication 100,Communication 1000,Communication 1001,Communication 1002,Communication 1003,Communication 1004,Communication 1005,Communication 1006,...,Word Processing 990,Word Processing 991,Word Processing 992,Word Processing 993,Word Processing 994,Word Processing 995,Word Processing 996,Word Processing 997,Word Processing 998,Word Processing 999
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.5,NaN,NaN,NaN,3.5
607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4.0,4.0,NaN,NaN


In [5]:
# workflowId = ratings.pivot_table(index='workflow', columns=['workflowTitle'], values='rating')
# workflowId

In [6]:
## fill remaining NaN with 0

user_ratings = user_ratings.dropna(thresh=0,axis=1).fillna(0)
user_ratings

workflowTitle,Communication 1,Communication 10,Communication 100,Communication 1000,Communication 1001,Communication 1002,Communication 1003,Communication 1004,Communication 1005,Communication 1006,...,Word Processing 990,Word Processing 991,Word Processing 992,Word Processing 993,Word Processing 994,Word Processing 995,Word Processing 996,Word Processing 997,Word Processing 998,Word Processing 999
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,3.5
607,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0


In [7]:
# workflowId = workflowId.dropna(thresh=0,axis=1).fillna(0)
# workflowId

# testing = user_ratings.to_dict('list')

# # user_ratings_unpivot = user_ratings.melt()

In [8]:
# workflowId.corr(user_ratings)

# # user_ratings_unpivot['value']

# testing

In [9]:
## Building Similarity Matrix using pearson correlation
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df

workflowTitle,Communication 1,Communication 10,Communication 100,Communication 1000,Communication 1001,Communication 1002,Communication 1003,Communication 1004,Communication 1005,Communication 1006,...,Word Processing 990,Word Processing 991,Word Processing 992,Word Processing 993,Word Processing 994,Word Processing 995,Word Processing 996,Word Processing 997,Word Processing 998,Word Processing 999
workflowTitle,,,,,,,,,,,,,,,,,,,,,
Communication 1,1.000000,-0.001642,-0.001642,-0.001642,-0.003693,0.331741,-0.001642,-0.001992,-0.001642,-0.002317,...,0.229216,-0.005304,-0.003809,-0.003600,-0.002242,0.209725,-0.005454,-0.007437,-0.001642,-0.003937
Communication 10,-0.001642,1.000000,-0.001642,-0.001642,-0.003693,0.247458,-0.001642,-0.001992,-0.001642,0.650072,...,-0.006042,-0.005304,-0.003809,-0.003600,-0.002242,-0.007801,-0.005454,-0.007437,-0.001642,-0.003937
Communication 100,-0.001642,-0.001642,1.000000,-0.001642,0.393901,-0.005389,-0.001642,0.970119,-0.001642,-0.002317,...,-0.006042,-0.005304,0.553794,-0.003600,-0.002242,-0.007801,-0.005454,0.189814,-0.001642,-0.003937
Communication 1000,-0.001642,-0.001642,-0.001642,1.000000,-0.003693,-0.005389,-0.001642,-0.001992,-0.001642,-0.002317,...,-0.006042,-0.005304,-0.003809,-0.003600,-0.002242,-0.007801,-0.005454,0.091188,-0.001642,-0.003937
Communication 1001,-0.003693,-0.003693,0.393901,-0.003693,1.000000,0.210914,-0.003693,0.397470,-0.003693,-0.005212,...,0.048665,0.013252,0.212769,-0.008098,-0.005043,0.004040,-0.012268,0.105067,-0.003693,-0.008856
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Word Processing 995,0.209725,-0.007801,-0.007801,-0.007801,0.004040,0.223786,-0.007801,-0.009463,-0.007801,-0.011008,...,0.265068,0.175008,0.133273,0.046018,-0.010651,1.000000,0.033474,0.149402,0.209725,0.255108
Word Processing 996,-0.005454,-0.005454,-0.005454,-0.005454,-0.012268,0.189179,0.267988,-0.006616,-0.005454,-0.007697,...,0.138708,0.073309,0.291793,-0.011957,0.405494,0.033474,1.000000,0.319328,-0.005454,-0.013076
Word Processing 997,-0.007437,-0.007437,0.189814,0.091188,0.105067,0.065497,-0.007437,0.182413,-0.007437,-0.010496,...,0.214573,0.118089,0.153560,0.098168,0.104101,0.149402,0.319328,1.000000,0.222689,-0.017832


In [10]:
# df_unpivot = user_ratings.melt()
# test_df = df_unpivot
# # test_df = test_df.drop(test_df.value == 0) 
# # test_df = df_unpivot.sample(n = 100836)
# test_df

In [11]:
joblib.dump(item_similarity_df,"item_similarity_df")

['item_similarity_df']

In [12]:
# user_corr = pd.Series('float64')

# userId=0

# for workflowTitle in user_ratings.iloc[userId].dropna().index:
#     corr_list = item_similarity_df[workflowTitle].dropna()*user_ratings.iloc[userId][workflowTitle]
#     user_corr = user_corr.append(corr_list)

In [13]:
# usercorr=user_corr.groupby(user_corr.index).sum()

In [14]:
# title_list = []
# for i in range(len(user_ratings.iloc[userId].dropna().index)):
#     if user_ratings.iloc[userId].dropna().index[i] in user_corr:
#         try:
#             title_list.append(user_ratings.iloc[userId].dropna().index[i])
#         except KeyError(0):
#             continue
#     else:
#         pass
# user_corr = user_corr.drop(title_list)
# title_list

In [15]:
# print("Hi, based on the workflow that you have used: \n")
# for i in user_ratings.iloc[userId].dropna().index[:3]:
#     print(i)
# print("\n I would suggest you to watch these films \n")
# for i in user_corr.index[1:4]:
#     print(i)

In [16]:
def get_similar_movies(movie_name,user_rating):
    similar_score = item_similarity_df[movie_name]*(user_rating-2.5)
    similar_score = similar_score.sort_values(ascending=False)
    
    return similar_score

In [17]:
# categories = [("Word Processing 1711",5), ("Word Processing 5", 5),
#                 ("Word Processing 19",3), ("Word Processing 700", 4), ("Word Processing 898", 4), ("Word Processing 123", 5)
#                     , ("Word Processing 434", 2), ("Word Processing 242", 5)
               
#                ]

categories = [("Communication 13",5),
                ("Communication 547",5),
                ("Communication 19",3), 
              ("Communication 709", 1), ("Communication 6", 3), ("Communication 1", 4), 
                  ("Communication 14", 4), ("Communication 1001", 5)
             ]



def check_seen(movie,seen_movies):
    for item,rating in seen_movies:
        if item == movie:
            return True
    
    return False


similar_movies = pd.DataFrame()

for movie,rating in categories:
    similar_movies = similar_movies.append(get_similar_movies(movie,rating),ignore_index=True)

all_recommend = similar_movies.sum().sort_values(ascending=False)
# all_recommend = similar_movies.sum().sort_values(ascending=False).head(108)
# for movie,score in all_recommend.iteritems():
#     if not check_seen(movie,categories):
#         print(movie + " \t score = "+ str(score))

count = 0
for movie,score in all_recommend.iteritems():
    if(count < 3):
        if not check_seen(movie,categories):
            print(movie + " \t score = "+ str(score))
            count += 1
        else:
            break

# store = []
# for movie,score in all_recommend.iteritems():
#     if not check_seen(movie,categories):
#         print(movie + " \t score = "+ str(score))
#         store.append(score)
        
# store

Communication 984 	 score = 3.8507438272508554
Communication 939 	 score = 3.8458657659664244
Communication 1267 	 score = 3.692737218802126


In [60]:
avg_rating = ratings.groupby('workflowTitle')['rating'].transform('mean')
avg_rating = avg_rating.head(100)
avg_rating

0     3.92093
1     3.92093
2     3.92093
3     3.92093
4     3.92093
       ...   
95    3.92093
96    3.92093
97    3.92093
98    3.92093
99    3.92093
Name: rating, Length: 100, dtype: float64

In [122]:
from pandas import DataFrame
import statsmodels.api as sm
import numpy as np

dfActual = DataFrame(ratings, columns=['workflowTitle','userId', 'rating','workflow'])
dfActual = avg_rating
actual = df['rating']

print_model = model.summary()
predictions = dfTest['rating']
actual = actual.iloc[:100:]
actual

0     4.0
1     4.0
2     4.5
3     2.5
4     4.5
     ... 
95    5.0
96    5.0
97    5.0
98    4.0
99    5.0
Name: rating, Length: 100, dtype: float64

In [123]:
from pandas import DataFrame
your_list = store
dfTest = DataFrame (your_list,columns=['rating'])
dfTest['rating']

0     4.723551
1     4.533751
2     4.532806
3     4.359381
4     4.330872
        ...   
95    3.408427
96    3.390973
97    3.387634
98    3.386539
99    3.384478
Name: rating, Length: 100, dtype: float64

In [124]:
print("MSE: ", recmetrics.mse(actual, predictions))
print("RMSE: ", recmetrics.rmse(actual, predictions))

MSE:  0.97785231827642
RMSE:  0.988864155623218
